## Notice:
Uncomment the next cell and run once if you have never 
set your Colab notebook work environment to do C, C++ and NVCC, GCC compiler.

# Prepare the Environment

In [ ]:
# Before starting execution of this cell, go to Menu bar and change the runtime type to GPU.
# Runtime>>Change runtime type>>Hardware accelerator>>GPU
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version

! apt-get install gcc-7 g++-7
!which gcc-7
! update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 70 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!gcc --version

# This is the starting point if the environment is already set up before.

In [1]:
# check if there is GPU in the device
!nvidia-smi

Fri Feb  3 16:14:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# always run this cell before using NVCC commands
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oa3gcevd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oa3gcevd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=efc1113328bcd626c912134af8522ec36614332fcfdff699024d97e8566c8e30
  Stored in directory: /tmp/pip-ephem-wheel-cache-31g0m9pd/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [7]:
# A quick check if everything working fine:
# if the output is 2 + 3 = 5 it means we can work with the GPU now

# %%cu Tells the compiler the following code is CUDA-C Program
%%cu  
#include <iostream>
__global__ void add(int a, int b, int *c) {
  *c = a + b;
}

int main() {
  int a = 2, b = 3, c;
  int *dev_c;
  cudaMalloc(&dev_c, sizeof(int));
  add<<<1, 1>>>(a, b, dev_c);
  cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("%d + %d = %d\n", a, b, c);
  cudaFree(dev_c);
  return 0;
}

2 + 3 = 5



# This is where the real execution of the project starts.

In [14]:
# connect to google drive Go to the directory where you have saved the files downloaded from github.
# Example: my files are in "/content/drive/MyDrive/CUDA"
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CUDA

# to check the current working directory uncomment the following line.
# !pwd

# sometimes the sbatch file have read only permission the following line will change the permistion to executable.
!chmod +x Merge.sbatch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CUDA


In [12]:
# you can run the .cu file as follows: 
# NOTE: Check the documentation on what parameters to use when you execute the compiled file.
! nvcc -o Merge_sort_1 Merge_sort_1.cu
! ./Merge_sort_1 1000 10 "false" "ture"

Saving unsorted array to Unsorted.txt file ...
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@ MergeSortWithCuda @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
First loop Configuration
Block/Grid: 10 
Thread/Block: 51 
Total threads: 510 
T_Time: 0.000652
@@@@@@@@@@@@@@@@@@@@@@@@
@@ SORTING IS CORRECT @@
@@@@@@@@@@@@@@@@@@@@@@@@
Saving sorted array to sorted.txt file ...


In [13]:
# Or run the sbatch file as follows
!./Merge.sbatch

Merge_Sort By Sami Osman
Array size:          1000
Block size:          10
Analysis flag        false
Shared memory flag:  false
Saving unsorted array to Unsorted.txt file ...
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@ MergeSortWithCuda @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
First loop Configuration
Block/Grid: 10 
Thread/Block: 51 
Total threads: 510 
T_Time: 0.000653
@@@@@@@@@@@@@@@@@@@@@@@@
@@ SORTING IS CORRECT @@
@@@@@@@@@@@@@@@@@@@@@@@@
Saving sorted array to sorted.txt file ...


# THE END :)
# THANK YOU!